In [2]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecVideoRecorder
from stable_baselines3.common.evaluation import evaluate_policy
import gymnasium
import ale_py
import os
import optuna 

In [3]:
def obj(trial):
    # Environment creation
    env = make_atari_env("SpaceInvadersNoFrameskip-v4",n_envs=4,seed=0)
    # Stack 4 frames
    env = VecFrameStack(env, n_stack=4)

    learning_starts = trial.suggest_int("learning_starts", 1000, 100_000)
    target_update_interval = trial.suggest_int("target_update_interval", 1, 1000)
    exploration_fraction = trial.suggest_float("exploration_fraction", 0.1, 0.5)
    exploration_final_eps = trial.suggest_float("exploration_final_eps", 0.01, 0.1)
    batch_size = trial.suggest_int("batch_size", 32, 128)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    gradient_steps = trial.suggest_int("gradient_steps", 1, 10)
    buffer_size = trial.suggest_int("buffer_size", 1000, 100_000)
    train_freq = trial.suggest_int("train_freq", 1, 10)

    # Create the agent and train it
    agent = DQN("CnnPolicy", env, verbose=1,buffer_size=buffer_size, learning_starts=learning_starts, train_freq=train_freq, 
                target_update_interval=target_update_interval, exploration_fraction=exploration_fraction, 
                exploration_final_eps=exploration_final_eps, batch_size=batch_size, learning_rate=learning_rate, 
                gradient_steps=gradient_steps, optimize_memory_usage=False)
    agent.learn(total_timesteps=10_000)

    mean_reward, _= evaluate_policy(agent, env, n_eval_episodes=10, deterministic=True)
    env.close()
    return mean_reward

In [4]:
study = optuna.create_study(direction="maximize")
study.optimize(obj, n_trials=100, n_jobs=1)

print("Best hyperparameters:", study.best_params)

[I 2024-11-27 09:43:41,746] A new study created in memory with name: no-name-90357ff5-cd61-4046-97e5-74c55de8c728
C:\Users\jamb2\AppData\Local\Temp\ipykernel_20116\3213680254.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)


Using cpu device
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.712    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 204      |
|    time_elapsed     | 7        |
|    total_timesteps  | 1512     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.92e+03 |
|    ep_rew_mean      | 128      |
|    exploration_rate | 0.638    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 223      |
|    time_elapsed     | 8        |
|    total_timesteps  | 1900     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.16e+03 |
|    ep_rew_mean      | 212      |
|    exploration_rate | 0.488    |
| time/               |          |
|    episodes         | 12      

[I 2024-11-27 09:44:56,159] Trial 0 finished with value: 18.5 and parameters: {'learning_starts': 32476, 'target_update_interval': 450, 'exploration_fraction': 0.4740410447777754, 'exploration_final_eps': 0.09688922499719223, 'batch_size': 99, 'learning_rate': 2.120879758601238e-05, 'gradient_steps': 10, 'buffer_size': 44364, 'train_freq': 9}. Best is trial 0 with value: 18.5.
C:\Users\jamb2\AppData\Local\Temp\ipykernel_20116\3213680254.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)


Using cpu device
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.701    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 194      |
|    time_elapsed     | 7        |
|    total_timesteps  | 1512     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.92e+03 |
|    ep_rew_mean      | 128      |
|    exploration_rate | 0.624    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 197      |
|    time_elapsed     | 9        |
|    total_timesteps  | 1900     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.16e+03 |
|    ep_rew_mean      | 212      |
|    exploration_rate | 0.468    |
| time/               |          |
|    episodes         | 12      

[I 2024-11-27 09:46:02,043] Trial 1 finished with value: 18.5 and parameters: {'learning_starts': 36166, 'target_update_interval': 659, 'exploration_fraction': 0.4600726076597029, 'exploration_final_eps': 0.0887979820602534, 'batch_size': 123, 'learning_rate': 1.3383483450661942e-05, 'gradient_steps': 7, 'buffer_size': 91095, 'train_freq': 4}. Best is trial 0 with value: 18.5.


...

Using cpu device
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.674    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 183      |
|    time_elapsed     | 8        |
|    total_timesteps  | 1512     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.92e+03 |
|    ep_rew_mean      | 128      |
|    exploration_rate | 0.59     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 187      |
|    time_elapsed     | 10       |
|    total_timesteps  | 1900     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.16e+03 |
|    ep_rew_mean      | 212      |
|    exploration_rate | 0.42     |
| time/               |          |
|    episodes         | 12      

[I 2024-11-27 11:34:52,203] Trial 98 finished with value: 257.5 and parameters: {'learning_starts': 76495, 'target_update_interval': 862, 'exploration_fraction': 0.450257808134455, 'exploration_final_eps': 0.028227367737560724, 'batch_size': 75, 'learning_rate': 0.00014709903881520415, 'gradient_steps': 2, 'buffer_size': 36234, 'train_freq': 3}. Best is trial 47 with value: 376.0.


Using cpu device
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.347    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 174      |
|    time_elapsed     | 8        |
|    total_timesteps  | 1512     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.92e+03 |
|    ep_rew_mean      | 128      |
|    exploration_rate | 0.18     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 179      |
|    time_elapsed     | 10       |
|    total_timesteps  | 1900     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.16e+03 |
|    ep_rew_mean      | 212      |
|    exploration_rate | 0.0472   |
| time/               |          |
|    episodes         | 12      

[I 2024-11-27 11:36:07,762] Trial 99 finished with value: 343.0 and parameters: {'learning_starts': 62085, 'target_update_interval': 835, 'exploration_fraction': 0.2207848438271597, 'exploration_final_eps': 0.04715976502721496, 'batch_size': 77, 'learning_rate': 5.6234627887521216e-05, 'gradient_steps': 8, 'buffer_size': 10413, 'train_freq': 5}. Best is trial 47 with value: 376.0.


Best hyperparameters: {'learning_starts': 71837, 'target_update_interval': 237, 'exploration_fraction': 0.2589892072881381, 'exploration_final_eps': 0.03801659272554749, 'batch_size': 117, 'learning_rate': 0.0007658002238434514, 'gradient_steps': 7, 'buffer_size': 2530, 'train_freq': 3}


 Best hyperparameters: {'learning_starts': 71837, 'target_update_interval': 237, 'exploration_fraction': 0.2589892072881381, 'exploration_final_eps': 0.03801659272554749, 'batch_size': 117, 'learning_rate': 0.0007658002238434514, 'gradient_steps': 7, 'buffer_size': 2530, 'train_freq': 3}

In [6]:
# Environment creation
env = make_atari_env("SpaceInvadersNoFrameskip-v4",n_envs=4,seed=0)
# Stack 4 frames
env = VecFrameStack(env, n_stack=4)

# Create the agent and train it
agent = DQN("CnnPolicy", env, verbose=1,buffer_size=2530, learning_starts=71837, train_freq=3, 
            target_update_interval=237, exploration_fraction=0.2589892072881381, exploration_final_eps=0.03801659272554749,
            batch_size=117, learning_rate=0.0007658002238434514, gradient_steps=7, optimize_memory_usage=False)
agent.learn(total_timesteps=10_000)

# Create a folder to save videos
video_folder = "videos/"
os.makedirs(video_folder, exist_ok=True)

# Wrap environment for video recording
recording_env = VecVideoRecorder(env, video_folder,
                                 record_video_trigger=lambda x: True,
                                 video_length=1000) # Record every 5000 steps
obs = recording_env.reset()

# Record video
for _ in range(1000):
    action, _ = agent.predict(obs, deterministic=True)
    obs, reward, done, info = recording_env.step(action)
    if done.all():
        obs = recording_env.reset()
recording_env.close() 

print(f"Video saved in {video_folder}")

Using cpu device
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.438    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 183      |
|    time_elapsed     | 8        |
|    total_timesteps  | 1512     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.92e+03 |
|    ep_rew_mean      | 128      |
|    exploration_rate | 0.294    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 182      |
|    time_elapsed     | 10       |
|    total_timesteps  | 1900     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.16e+03 |
|    ep_rew_mean      | 212      |
|    exploration_rate | 0.038    |
| time/               |          |
|    episodes         | 12      

MoviePy - Done !
MoviePy - video ready c:\workspace\RL-Fun\videos\rl-video-step-0-to-step-1000.mp4
Video saved in videos/
